In a previous post, I described a procedure for sampling from complicated probability distributions.
There's a beautiful and decades-old theory for how to sample from probability distributions defined over finite-dimensional vector spaces using Monte Carlo methods.
One of the subtle points that I tried to address there is what happens when the quantity we want to sample is a function of space and time.
For example, we might want to infer the conductivity of an aquifer from well head measurements; the material density in the earth's subsurface from satellite gravimetry; or basal drag between a glacier and the landscape from measurements of its surface velocity.
The parameter space is now a space of functions, and so it has infinitely many dimensions.
Extending common algorithms to work in function spaces is hard.

Since writing that first post, I read [this paper](https://doi.org/10.1214/13-STS421) by Cotter and others.
I think my previous post was wrong in a few respects now.
Here I'd like to explain why (it involves Weyl's law) and how to fix it (it involves Nitsche's method).

The failure is trying to bolt a statistical interpretation onto deterministic inverse problems.
I'm guilty of this.
I'd argue that many textbooks on inverse problems are too.
One misstep is to assume that common choices of regularization functional used in the deterministic inverse problems literature are going to translate into sensible prior distributions.
They don't.

To clarify the notation for later, the quantity we're interested in is a field $q$ defined over some spatial domain $\Omega$.
This field lives in some separable Hilbert space $H$, which is most likely the space of square-integrable functions $L^2(\Omega)$.
But it could also be the Sobolev space $W_1^2(\Omega)$ or something else.
The field is assumed to take random values determined by some probability distribution $\pi(q)$.
In a deterministic inverse problem, we would add a multiple of a functional $R(q)$ to the objective in order to regularize the problem.
The conventional approach is to use a multiple of
$$R(q) = \frac{1}{2}\int_\Omega|\nabla q|^2dx.$$
We can make this look like a statistical inference problem by taking the prior probability distribution $\rho$ to have
$$-\ln \rho(q) \propto R(q) + \ldots$$
The ellipses denote (in the quantum mechanic's parlance) another infinite constant that we can set to zero.

For the rest of this post, we'll be concerned exclusively with the case where $\pi$ is a Gaussian measure.
It's conventional to describe a Gaussian measure by its mean $m$ and its covariance operator $C$.
In our case, however, we'll work instead with the *precision* operator $L$, which is equal to $C^{-1}$, the inverse of the covariance matrix.
For problems posed in function spaces, the precision operator is often a linear differential operator with a concise expression.
As we'll see shortly, the covariance matrix has to be a compact operator, which will mean that $L$ can be unbounded.
In future posts we'll look at what happens when $\pi$ is not Gaussian.

### Proper priors

In the deterministic inverse problems literature, it's common to use
$$R(q) = \frac{1}{2}\int_\Omega|\nabla q|^2dx$$
as a regularization functional.
This accomplishes several goals.
First, the inferred parameters obtained without regularization usually have spurious high-wavenumber noise.
Adding this penalty filters out the noise.
Second, it can guarantee a unique solution where the problem would be otherwise ill-posed.

If we try to form a Gaussian measure $\rho$ for which $-\ln\rho \propto R(q) + \ldots$, we'll run aground right away.
Unless we also add boundary conditions to $q$, there is no constraint on the average value of $q$.
We can add any constant we want to $q$ and the value of $R$ is unchanged.
Viewing this in a probabilistic light now, the putative prior distribution doesn't integrate to 1 -- it is *improprer*.

Strictly speaking, you can use an improper prior so long as the posterior is proper.
The observational data need to constrain the constant mode and this does happen.
So we would use an improper prior when we know nothing at all about what value the parameter can take and we expect that the data we have can give us that informaiton.

Do we really have no prior information whatsoever about the average value of the parameters we want to infer?
Let's take the example I mentioned above about inferring the density of material within the earth from satellite observations of earth's gravitational pull.
Maybe you don't remember common rock densities off the top of your head, but you can go fetch one and weigh it and then put it in a beaker of water.
I'll save you the trouble, it's probably got a density around 3000 kg/m${}^3$.
Granted, the densities deep inside the earth are higher.
Say you guessed that the plausible range spanned an order of magnitude in both directions: between 300 and 30,000 kg/m${}^3$.
You wouldn't be doing particularly good but you also wouldn't be doing terrible either.
So it beggars belief that we should use a prior for a geophysical inverse problem that is totally uninformative about the mean value.
It might have high variance.
It shouldn't be infinite.

### Trace class operators

To prescribe a Gaussian measure, we need to know its mean $m$ and its precision operator $L$.
The precision has to be symmetric and positive-definite.
My objection above about properness of the prior is equivalent to the statement that $L$ has a zero eigenvalue.
In the function space setting, there are more criteria.
We don't think about them in finite dimensions because they're almost vacuously true.
In the infinite-dimensional case they demand some thought (ugh).

The first criterion is that the sum of all the eigenvalues of the covariance operator, i.e. the trace, is finite.
In finite dimensions this is obvious.
Being a trace class operator on a function space is special.
For example, the identity operator or any unitary map is not trace class.
Since we've chosen to work with the precision operator instead of the covariance, we can rephrase this condition as saying that the sum of the reciprocals of the eigenvalues of the precision has to be finite.

#### Two choices of precision operator

Suppose we wanted to do the bare minimum of effort to turn a determinstic inverse problem into a statistical one.
Rather than use the improprer prior I showed above, we could instead use
$$R(q) = \frac{1}{2}\int_\Omega\left(q^2 + \gamma^2|\nabla q|^2\right)dx$$
where $\alpha$ is some length scale we have to choose.
The precision operator is then
$$L = I - \gamma^2\Delta,$$
and we can write the more abstract form $R(q) = \frac{1}{2}\langle Lq, q\rangle$.
This choice of precision operator gets rid of the zero eigenvalue from before, so it gives us a proper prior.
Is it true that $\text{Tr}(L^{-1}) < \infty$ for this choice of $L$?

We can answer that question using Weyl's law.
I want to be careful here about the units, so I'll adopt a slightly different convention and say that $\lambda$, $\phi$ are an eigenvalue / eigenfunction pair for $-\Delta$ if
$$-\Delta\phi = \lambda^{-2}\phi.$$
This choice, instead of the more conventional one, makes $\lambda$ have units of length.
Weyl's law, as originally stated, gives an expression for the eigenvalue counting function:
$$N(\lambda) \equiv \#\{\text{eigenvalues of }-\Delta\text{ greater than }\lambda\} \sim \text{const}\times\text{vol}(\Omega)\times\lambda^{-d}.$$
If all you knew was that the asymptotics of the eigenvalue counting function have something to do with the volume of the domain (can you hear the size of the drum?) and that it has to go to infinity as $\lambda$ goes to 0, you could guess at this purely from dimensional analysis.
Figuring out what the constant prefactor and the remainder term are is harder but we won't need that much detail.

Weyl's law then implies that the eigenvalues decay like
$$\lambda_n \sim \text{const}\times\text{vol}(\Omega)^{1/d}\times n^{-1/d}.$$
So the eigenvalues of $(I - \gamma^2\Delta)^{-1}$ go like
$$\sigma_n = (1 + \gamma^2\lambda_n^{-2})^{-1} \sim \text{const}\times\frac{\gamma^2}{\text{vol}(\Omega)^{2/d}} \times n^{-2/d}.$$
In dimension $d = 2$ the eigenvalues decay like $n^{-1}$.
If we sum these, the trace diverges like the harmonic series.
In 3D the divergence is even more rapid.

Suppose we instead take the precision to look like the biharmonic operator plus lower-order terms:
$$L = I - \alpha^2\Delta + \gamma^4\Delta^2$$
where now we need to pick two length scales $\alpha$, $\gamma$.
Weyl's law now comes to the rescue to show that the eigenvalues of $L^{-1}$ are instead asymptotic to
$$\sigma_n \sim \text{const} \times \frac{\gamma^4}{\text{vol}(\Omega)^{4/d}} \times n^{-4/d}$$
which is summable in dimensions 2 and 3.

#### Numerical demonstration

Let's write some code to illustrate this.
If we have a Gaussian random variable $\xi$ with mean 0 and we multiply it by a matrix $\Gamma$, the covariance of $\Gamma\xi$ is
$$\text{cov}(\Gamma\xi) = \Gamma\,\text{cov}(\xi)\,\Gamma^*.$$
So if we want to make a random variable that has $L$ as its precision operator, we can do that by computing some kind of matrix square root of $L$.
We can do that either with spectral theory or computing the Cholesky decomposition.
The code below uses the Cholesky decomposition.
But it's harder to scale Cholesky or other factorization approaches to smaller meshes for 3D problems and in that setting we would be more likely to use spectral theory approaches.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import firedrake
from firedrake import (
    Constant, exp, inner, outer, avg, jump, grad, dx, ds, dS, assemble
)
from firedrake.petsc import PETSc

In [ ]:
nx, ny = 32, 32
lx, ly = 1.0, 1.0
Lx, Ly = Constant(lx), Constant(ly)
mesh = firedrake.RectangleMesh(nx, ny, lx, ly, diagonal="crossed")
Q = firedrake.FunctionSpace(mesh, "CG", 2)
area = assemble(Constant(1) * dx(mesh))

This code is lifted directly from the previous notebook.

In [ ]:
class NoiseGenerator:
    def __init__(
        self,
        function_space,
        covariance=None,
        generator=np.random.default_rng()
    ):
        if covariance is None:
            ϕ = firedrake.TrialFunction(function_space)
            ψ = firedrake.TestFunction(function_space)
            covariance = inner(ϕ, ψ) * dx

        M = assemble(covariance, mat_type='aij').M.handle
        ksp = PETSc.KSP().create()
        ksp.setOperators(M)
        ksp.setUp()

        pc = ksp.pc
        pc.setType(pc.Type.CHOLESKY)
        pc.setFactorSolverType(PETSc.Mat.SolverType.PETSC)
        pc.setFactorSetUpSolverType()
        L = pc.getFactorMatrix()
        pc.setUp()

        self.rng = generator
        self.function_space = function_space
        self.preconditioner = pc
        self.cholesky_factor = L

        self.rhs = firedrake.Function(self.function_space)
        self.noise = firedrake.Function(self.function_space)

    def __call__(self):
        z, ξ = self.rhs, self.noise
        N = len(z.dat.data_ro[:])
        z.dat.data[:] = self.rng.standard_normal(N)

        L = self.cholesky_factor
        with z.dat.vec_ro as Z:
            with ξ.dat.vec as Ξ:
                L.solveBackward(Z, Ξ)
                Ξ *= np.sqrt(area / N)

        return ξ.copy(deepcopy=True)

Here we'll make an object to sample from the first random process, which uses the precision $I - \gamma^2\Delta$.
The operator $L^{-1}$ is not trace-class.

In [ ]:
ϕ, ψ = firedrake.TestFunction(Q), firedrake.TrialFunction(Q)
γ = firedrake.sqrt(Lx * Ly)
M = (ϕ * ψ + γ**2 * inner(grad(ϕ), grad(ψ))) * dx

In [ ]:
h1_generator = NoiseGenerator(
    function_space=Q,
    covariance=M,
    generator=np.random.default_rng(1453),
)

Now we'd like to use the biharmonic functional
$$R(q) = \int_\Omega\left(q^2 + \gamma^4|\nabla^2q|^2\right)dx,$$
which penalizes large values of the curvature $\nabla^2q$.
Conventional finite element basis functions are continuous and piecewise-differentiable, but their derivatives have jump discontinuities across cell boundaries.
There are continuously-differentiable finite element bases which we could use to construct a conforming discretization of the curvature penalty.
I'll instead use a non-conforming discretization based on ordinary CG elements.
This approach is similar to how we used DG elements for the convection-diffusion equation.
For that problem, we applied Nitsche's method at all of the cell boundaries in order to make the solution continuous.
Here we'll instead apply Nitsche's method at all the cell boundaries to make the solution's gradient continuous.
I'm partly following [this paper](https://doi.org/10.1515/jnma-2023-0028) for discretization of the curvature penalty but working back to a minimization form.

In [ ]:
ϕ = firedrake.Function(Q)
Dϕ = grad(ϕ)
DDϕ = grad(Dϕ)

h = firedrake.FacetArea(mesh)
vol = firedrake.CellVolume(mesh)

α = firedrake.Constant(4.0)
k = firedrake.Constant(Q.ufl_element().degree())
β = 3 * α * k * (k - 1) / 8 * avg(h)**2 * avg(1 / vol)
β_Γ = 3 * α * k * (k - 1) * h**2 / vol

ν = firedrake.FacetNormal(mesh)
J_cells = inner(DDϕ, DDϕ) * dx
J_facets = avg(inner(DDϕ, outer(ν, ν))) * jump(Dϕ, ν) * dS
J_facet_penalty = β / avg(h) * jump(Dϕ, ν)**2 * dS
J_boundary = inner(Dϕ, ν) * inner(DDϕ, outer(ν, ν)) * ds
J_boundary_penalty = β_Γ / h * inner(Dϕ, ν)**2 * ds

J_Δ = 0.5 * (J_cells - J_facets - J_boundary + J_facet_penalty + J_boundary_penalty)
J_2 = 0.5 * ϕ**2 * dx

J = J_2 + γ**4 * J_Δ

In [ ]:
from firedrake import derivative

M = derivative(derivative(J, ϕ), ϕ)
h2_generator = NoiseGenerator(
    function_space=Q,
    covariance=M,
    generator=np.random.default_rng(seed=1666),
)

The plots below show samples from the precision operator $I - \gamma^2\Delta$ first and $I + \gamma^4\Delta^2$ second.
Maybe the most compelling argument I can give you for why biharmonic regularization is the way to go is, just look at it.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, subplot_kw={"projection": "3d"})
for ax in axes.flatten():
    w = h1_generator()
    firedrake.trisurf(w, axes=ax)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, subplot_kw={"projection": "3d"})
for ax in axes.flatten():
    z = h2_generator()
    firedrake.trisurf(z, axes=ax)

Using the biharmonic operator is much less common than the Laplacian.
I have a hunch that this is because it's more difficult to implement and not because it doesn't give better results.
You can pick good values of the penalty parameters based on the mesh quality and polynomial degree -- you don't need to, for example, iterate on them based on candidate solutions.
But this knowledge isn't disseminated very well.

### Proposal mechanism

Questions about what kinds of covariance operator to choose concern the problem that we wish to solve.
The final challenge I ran into is not about what problem to solve but how to solve it.
We've only talked so far about what prior to use and not about Markov chain Monte Carlo methods as such.
By way of recap, what an MCMC algorithm needs is a *proposal generation kernel* describing how to generate a new candidate state $q_2$ from the current state $q_1$:
$$Q(q_1\rightarrow q_2) = \text{probability of proposing }q_2\text{ given the current state }q_1.$$
At each step, we generate a new candidate and either accept or reject it according to some rule
$$\alpha(q_1\rightarrow q_2) = \text{probability of accepting }q_2\text{ given the current state }q_1.$$
The most common rule is Metropolis-Hastings.
There are other possible rules, like the [Barker rule](https://doi.org/10.1111/rssb.12482).
The combination of a good proposal mechanism and an accept-reject step gives a stochastic process with the desired distribution as its unique stable steady state.

All of the advanced MCMC sampling algorithms work by using more and more sophisticated proposal generation kernels.
A common thread is to use a Langevin-type equation to generate samples:
$$\dot q = -\frac{1}{2}K^{-1}\nabla\log\pi + \dot w$$
where $K$ is some s.p.d. linear operator.
The noise forcing has
$$\text{cov}(w(s), w(t)) = \delta(s - t)K^{-1}.$$
Discretize the ODE however you like and you get a proposal mechanism.
Seems fine but isn't.
In infinite dimensions, we can only use one discretization scheme.
To understand why, we need to think (ugh) about measure theory (yech).

#### Absolute continuity of proposal mechanism

If we want to generate samples from a distribution $\pi$ and we've chosen a proposal kernel $Q$, the Metropolis-Hastings accept-reject formula is
$$\alpha(q_1 \rightarrow q_2) = \min\left\{1,\frac{\pi(q_2)Q(q_2\rightarrow q_1)}{\pi(q_1)Q(q_1\rightarrow q_2)}\right\}.$$
(As an aside, I never really understood this on a gut level until I read [this paper](https://www.jstor.org/stable/3182774).)
There are technical requirements on the proposal mechanism that we get to ignore in finite dimensions but have to pay attention to in function spaces.
For example, in finite dimensions we can often take $Q$ to be a normal random variable with mean $q_1$.
This proposal might converge so slow as to be useless for practical purposes, but it won't break the theory.
In infinite dimensions, reasonable-looking proposals break the theory.

The technical requirements on the proposal mechanism are laid out most clearly in [Tierney (1998)](https://www.jstor.org/stable/2667233).
First, note that the proposal generation kernel is a mapping from the current state $q_1$ to a probability density w.r.t. $q_2$.
It isn't a probability density as such, only a mapping whose outputs are probability densities.
Now we'll define a probability density over pairs $q_1$, $q_2$:
$$R(q_1\rightarrow q_2) = \pi(q_1)Q(q_1\rightarrow q_2).$$
This density describes the probability, assuming that we had converged to the true equilibrium density, of starting at $q_1$ and proposing $q_2$.
Again, this is a joint density over both $q_1$ and $q_2$, not a mere mapping from $q_1$ to a density over $q_2$.
Mind you, we don't have access to $\pi$ as such -- that was the whole point of the exercise.

Now we can ask what the relationship is between the forward and the *reverse* density
$$R(q_1 \leftarrow q_2) = R(q_2 \rightarrow q_1).$$
This describes the probability of starting out instead at $q_2$ and proposing $q_1$.
Remember that a Markov chain in statistical equilibrium settles into a state of detailed balance -- the probability of starting at one state and ending up at another is the same as going backwards.

Here's the technical condition which is so hard to achieve in practice and which Tierney proves.
The proposal mechanism $Q$ does not need to be exactly in detailed balance -- after all, that's what the accept-reject mechanism is supposed to do for us.
**But the forward proposal density cannot, with positive probability, suggest a new state where the reverse move has zero probability under the reverse proposal density.**

There's fancy measure theory jargon for the condition I'm talking about: the proposal density needs to be [absolutely continuous](https://en.wikipedia.org/wiki/Absolute_continuity) with respect to the reverse proposal.
In the finite-dimensional case, it's hard to even imagine how you could fuck up so badly that your proposal mechanism has zero probability of inhabiting part of the state space.
A pure random walk -- taking the proposal to be a standard normal random variable -- won't fall into this trap.
It might converge real slow, but it can at least get from anywhere to anywhere.

We saw earlier that making sure the covariance is a trace-class operator is not a guaranteed proposition.
You might guess that finding a forward proposal generation kernel that isn't singular w.r.t. its reverse is also not guaranteed.
You'd be right.
In the function space setting, *almost no proposal mechanisms are absolutely continuous*.

#### Absolute continuity of Gaussian measures

We need to unpack the [Feldman-Hájek theorem](https://en.wikipedia.org/wiki/Feldman%E2%80%93H%C3%A1jek_theorem) in order to understand this.
It's kind of a doozy and I don't find the form that it's usually stated in to be easy to grasp at first sight.

A Gaussian measure is defined entirely by its mean and precision operator.
We'll assume that the means are zero in order to simplify things and because we won't need to consider non-zero means.
Suppose we have two Gaussian measures on $H$ with precision operators $L$ and $K$.
The precision operators have eigendecompositions; the eigenfunctions won't matter because we can map one basis into another with a unitary operator.
We'll let $\sigma_n(L)$, $\sigma_n(K)$ be the $n$-th eigenvalue of each operator.
The Feldman-Hájek theorem tells us that the two measures are mutually equivalent if and only if
$$\sum_n\left(\frac{\sigma_n(K)}{\sigma_n(L)} - 1\right)^2 < \infty.$$
Otherwise, the two measures are mutually singular.
In short, the two eigenvalue sequences need to grow at the same rate even including the same constant multiplicative factor.
For example, suppose that $K = a\cdot L$ for some scalar $a$ that's not equal to 1.
Then $\sigma_n(K)/\sigma_n(L) - 1 = a - 1$.
For random variables taking values with values in a function space, the two distributions are mutually singular.
This is not like the finite-dimensional case at all.

#### MCMC in function spaces

Now let's discretize the Langevin equation.
We'll assume again that $-\log\pi = \frac{1}{2}\langle Lq, q\rangle$.
The simplest family of schemes we might come up with are the $\theta$ schemes:
$$\frac{q_{n + 1} - q_n}{\delta t} = -\frac{1}{2}K^{-1}L\left((1 - \theta)q_n + \theta\,q_{n + 1}\right) + \delta t^{-1/2}\,\delta w_n.$$
Rearranging terms, we get
$$q_{n + 1} = \left(I + \frac{\theta\,\delta t}{2}K^{-1}L\right)^{-1}\left\{\left(I - \frac{(1 - \theta)\delta t}{2}K^{-1}L\right)q_n + \delta t^{-1/2}\,\delta w_n\right\}.$$
That's kind of a lot.
We can learn something useful by assuming that $K = L$, i.e. that we can exactly invert the precision operator.
In this case, we get
$$q_{n + 1} = \frac{1 - (1 - \theta)\delta t/2}{1 + \theta\,\delta t/2}q_n + \frac{\sqrt{\delta t}}{1 + \theta\,\delta t/2}\delta w_n.$$
(Remember that now $\text{cov}(\delta w_n) = L^{-1}$.)
The proposal is a linear combination of Gaussian random variables, so it's also Gaussian and we can ask what its covariance is:
$$\text{cov}(q_{n + 1}) = \frac{(1 - (1 - \theta)\delta t / 2))^2}{(1 + \theta\,\delta t/2)^2}\text{cov}(q_n) + \frac{\delta t}{(1 + \theta\,\delta t/2)^2}\text{cov}(\delta w_n) = \ldots$$
We know from the outset that $\text{cov}(\delta W_n) = L^{-1}$.
Now suppose that the chain had converged, in which case $\text{cov}(q_n) = L^{-1}$ as well.
A bit of routine algebra gives
$$\ldots = \frac{(1 + \theta\,\delta t / 2)^2 + (1 - 2\theta)\,\delta t^2 / 4}{(1 + \theta\,\delta t/2)^2}L^{-1}.$$
When $\theta = 1/2$, this all reduces to $L^{-1}$.
For any other value of $\theta$, the covariance of the proposal is some non-trivial scalar multiple of $L^{-1}$.
The Feldman-Hájek theorem then tells us that the forward proposal density $R(q_1\rightarrow q_2)$ is singular w.r.t. its reversal $R(q_1 \leftarrow q_2)$.
The theory of MCMC on general state spaces then tells us that the resulting Markov chain would not converge.

In short, **only $\theta = 1/2$ can give a convergent Markov chain.**
If we do take $\theta = 1/2$, we can define
$$\alpha = \frac{1 - \delta t/4}{1 + \delta t / 4},$$
in which case we can concisely express the proposal generation mechanism as
$$q_{n + 1} = \alpha q_n + (1 - \alpha^2)^{1/2}\,\delta w_n$$
where again the $\delta w_n$ are uncorrelated Gaussian random variables with $\text{cov}(\delta w_n) = L^{-1}$.
We used an unholy mess of sophisticated math and bandied about some scary words like "Langevin".
But in the end all we did was reinvent an [AR(1) process](https://en.wikipedia.org/wiki/Autoregressive_model).
Embarrassing.

We made the simplifying assumption earlier that we could take our preconditioning operator $K$ to be equal to the exact precision $L$.
What if we don't want to assume that anymore?
To make some of the notation easier, introduce the matrix
$$S = I + \frac{\theta\,\delta t}{2}K^{-1}L.$$
If you work through some heinous matrix algebra, you can show that
$$\text{cov}(q_{n + 1}) = L^{-1} + \frac{(1 - 2\theta)\,\delta t^2}{4}S^{-1}K^{-1}LK^{-*}S^{-*}.$$
All of this simplifies to $L^{-1}$ if $\theta = 1/2$.
Otherwise, it's not equal to $L^{-1}$ and we run the risk of getting a singular proposal generation mechanism again.

### Practice

Before, we assumed that we could compute a matrix square root of $L$.
In 2D we can do this using the Cholesky factorization, but in 3D we would need iterative methods.
Suppose we considered it impossible or too aggravating to write a procedure to apply $L^{-1/2}$.
For example, say we insisted on using $L = I + \gamma^4\Delta^2$.
We could get even more exotic and include boundary terms:
$$\langle Lq, q\rangle = \int_\Omega\left(q^2 + \gamma^4|\nabla^2q|^2\right)dx + \kappa\int_{\partial\Omega}q^2ds.$$
This isn't equal to a square of operators like $(I - \gamma^2\Delta)^2$ -- that would be too convenient.
We could take
$$G = I - \gamma^2\Delta$$
and then use
$$K = G^2.$$
Note that $G$ is self-adjoint.
This operator isn't equal to $L$ but it does have the same spectral asymptotics.
The lower-order differential operators and the boundary terms don't matter.
We then generate proposal through
$$\begin{align}
z & = \left(I - \frac{\delta t}{4}G^{-2}L\right)q_n + \delta t^{-1/2}G^{-1}\xi_n \\
q_{n + 1} & = \left(I + \frac{\delta t}{4}G^{-2}L\right)^{-1}z
\end{align}$$
where the vector $\xi_n$ are i.i.d. standard normals.
There are lots of applications of $G^{-1}$ here.
As a first pass, we can do all these sequentially.
If you wanted to optimize this, you can solve a linear equation with the same matrix but multiple right-hand sides faster than doing repeated solves with a single right-hand side.
Since $G$ and $L$ are all symmetric and positive-definite and $G^2$ and $L$ are spectrally equivalent, we expect that the operator $I + \delta t\,G^{-2}L/4$ is going to be relatively easy to invert.